In [61]:
import pandas as pd
# Use a pipeline as a high-level helper
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from tqdm import tqdm


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [51]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct", max_new_tokens=2048)
pipe(messages)

Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'}]}]

In [8]:
def call_pipe(pipe, section_name, content):
    messages = [{"role": "system", "content": 
                 "You are an AI assistant who will assess whether a given text is used as a background, result or method section in a scientific paper."
                 "You will be given a section name and the text, and you will need to classify the text as one of [background, result, method]."
                 "You will also need to provide a reasoning for your classification."
                 "The output should strictly be a json with the following two keys: classification, reasoning."
                 "Example output would look like: {\"classification\": \"result\", \"reasoning\": \"This is the reasoning\"}"
    }, 
    {"role": "user", "content": f"section name: {section_name}, text: {content}"}]
    response = pipe(messages)[0]
    try:
        if response["generated_text"][2]["content"]:
            #print(response["generated_text"][2]["content"])
            content = response["generated_text"][2]["content"]
            start = content.find("{")
            end = content.rfind("}") + 1
            json_str = content[start:end]
            answers = json.loads(json_str)
            classification = answers["classification"]
            reasoning = answers["reasoning"]
            return classification, reasoning
        else:
            return "Invalid", "No reasoning provided"
    except Exception as e:
        print(e)
        return "Invalid", "No reasoning provided"

In [55]:

result = call_pipe(pipe, "Introduction", "However, how frataxin interacts with the Fe-S cluster biosynthesis components remains unclear as direct one-to-one interactions with each component were reported (IscS [12,22], IscU/Isu1 [6,11,16] or ISD11/Isd11 [14,15]).")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{"classification": "result", "reasoning": "This is the reasoning. The given text mentions specific components (IscS, IscU/Isu1, ISD11/Isd11) that are known to interact with the Fe-S cluster biosynthesis, but does not provide a direct description of how frataxin interacts with these components. The text suggests that the interaction is complex and may involve multiple components, but does not provide any specific details about the mechanism or outcome of this interaction."}


In [5]:
print(result)

('background', 'The text establishes the initial gap in knowledge regarding the interaction between frataxin and the Fe-S cluster biosynthesis components, setting the stage for subsequent research.')


In [67]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [68]:
data = preprocess("./data/train.jsonl")

In [64]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [ ]:
labels = []
reasonings = []
raw_output = []
ids = []

for i in tqdm(range(1365)):
    current_data = data.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_pipe(pipe, current_data.sectionName, current_data.string)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)

  0%|          | 3/1365 [00:13<1:41:58,  4.49s/it]

In [ ]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("results/Gemma/Gemma-1b/first-partition.csv")

In [80]:
filename = "r1_third_part.txt"
c = []
r = []
i = [third_partition.iloc[x].unique_id for x in range(len(third_partition))]

with open(filename) as file:
    lines = [line.rstrip() for line in file]
    for line in lines:
        if "ChatCompletion" in line:
            continue
        start = line.find("{")
        end = line.rfind("}") + 1
        if start == -1:
            continue
        json_str = json.dumps(eval(line[start:end]))
        answers = json.loads(json_str)
        
        classification = answers["classification"]
        reasoning = answers["reasoning"]
        print(classification)
        print(reasoning)
        c.append(classification)
        r.append(reasoning)

df = pd.DataFrame(zip(i, c, r), columns=["id", "model_classification", "reasoning"])

df.to_csv("deepseek_third_part.csv")

background
The citation discusses previous descriptions and suitability of PAR, which is providing context and existing knowledge, typical of a background section.
method
The text describes the use of the CatWalk system for gait analysis, detailing the experimental procedure and timing, characteristic of a methods section.
result
The citation references prior studies demonstrating high methane emissions, supporting findings in a discussion by aligning with established results.
background
The citation provides context on genetic interactions in polyploids by citing reviews, establishing foundational knowledge for the discussion.
result
The cited text refers to a figure showing performance improvements, indicating prior results used to justify the current methodology.
background
The citation summarizes prior findings on Emab's effects, providing context for the study's background.
method
The text details ERG recording protocols adhering to standards, describing experimental methods.
meth

In [69]:
openai = pd.read_csv("./4o_full.csv")
deepseek = pd.read_csv("./your_partition.csv")
train = data

In [71]:
deepseek_w_openai_c = 0
deepseek_c_openai_w = 0;
deepseek_w_openai_w = 0
deepseek_c_openai_c = 0
for i in range(len(deepseek)):
    index = i + 1365
    correct_label = train.iloc[index].label
    deepseek_correct = deepseek.iloc[i].model_classification == correct_label
    openai_correct = openai.iloc[index].model_classification == correct_label
    if deepseek_correct and openai_correct:
        deepseek_c_openai_c += 1
    elif deepseek_correct:
        deepseek_c_openai_w += 1
    elif openai_correct:
        deepseek_w_openai_c += 1
    else:
        deepseek_w_openai_w += 1

print(f"Deepseek is correct and openai is correct: {deepseek_c_openai_c}")
print(f"Deepseek is correct and openai is wrong: {deepseek_c_openai_w}")
print(f"Deepseek is wrong and openai is correct: {deepseek_w_openai_c}")
print(f"Deepseek is wrong and openai is wrong: {deepseek_w_openai_w}")

Deepseek is correct and openai is correct: 809
Deepseek is correct and openai is wrong: 78
Deepseek is wrong and openai is correct: 204
Deepseek is wrong and openai is wrong: 268
